In [1]:
import mwclient  # библиотека для работы с MediaWiki API для загрузки примеров статей Википедии
import mwparserfromhell  # Парсер для MediaWiki
import openai  # будем использовать для токинизации
import pandas as pd  # В DataFrame будем хранить базу знаний и результат токинизации базы знаний
import re  # для вырезания ссылок <ref> из статей Википедии
import tiktoken  # для подсчета токенов
from typing import List, Set, Tuple

In [2]:
# Соберем заголовки всех статей
def titles_from_category(
    category: mwclient.listing.Category, # Задаем типизированный параметр категории статей
    max_depth: int # Определяем глубину вложения статей
) -> Set[str]:
    """Возвращает набор заголовков страниц в данной категории Википедии и ее подкатегориях."""
    titles = set() # Используем множество для хранения заголовков статей
    for cm in category.members(): # Перебираем вложенные объекты категории
        if type(cm) == mwclient.page.Page: # Если объект является страницей
            titles.add(cm.name) # в хранилище заголовков добавляем имя страницы
        elif isinstance(cm, mwclient.listing.Category) and max_depth > 0: # Если объект является категорией и глубина вложения не достигла максимальной
            deeper_titles = titles_from_category(cm, max_depth=max_depth - 1) # вызываем рекурсивно функцию для подкатегории
            titles.update(deeper_titles) # добавление в множество элементов из другого множества
    return titles

In [9]:
CATEGORY_TITLE = "Category:Automotive industry"
WIKI_SITE = "en.wikipedia.org"

# Инициализация объекта MediaWiki
# WIKI_SITE ссылается на англоязычную часть Википедии
site = mwclient.Site(WIKI_SITE)

# Загрузка раздела заданной категории
category_page = site.pages[CATEGORY_TITLE]


In [10]:

# Получение множества всех заголовков категории с вложенностью на один уровень
titles = titles_from_category(category_page, max_depth=1)


print(f"Создано {len(titles)} заголовков статей в категории {CATEGORY_TITLE}.")

Создано 803 заголовков статей в категории Category:Automotive industry.


In [11]:
SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]


In [12]:
# Функция возвращает список всех вложенных секций для заданной секции страницы Википедии

def all_subsections_from_section(
    section: mwparserfromhell.wikicode.Wikicode, # текущая секция
    parent_titles: List[str], # Заголовки родителя
    sections_to_ignore: Set[str], # Секции, которые необходимо проигнорировать
) -> List[Tuple[List[str], str]]:
    """
    Из раздела Википедии возвращает список всех вложенных секций.
    Каждый подраздел представляет собой кортеж, где:
      - первый элемент представляет собой список родительских секций, начиная с заголовка страницы
      - второй элемент представляет собой текст секции
    """

    # Извлекаем заголовки текущей секции
    headings = [str(h) for h in section.filter_headings()]
    title = headings[0]
    # Заголовки Википедии имеют вид: "== Heading =="

    if title.strip("=" + " ") in sections_to_ignore:
        # Если заголовок секции в списке для игнора, то пропускаем его
        return []

    # Объединим заголовки и подзаголовки, чтобы сохранить контекст для chatGPT
    titles = parent_titles + [title]

    # Преобразуем wikicode секции в строку
    full_text = str(section)

    # Выделяем текст секции без заголовка
    section_text = full_text.split(title)[1]
    if len(headings) == 1:
        # Если один заголовок, то формируем результирующий список
        return [(titles, section_text)]
    else:
        first_subtitle = headings[1]
        section_text = section_text.split(first_subtitle)[0]
        # Формируем результирующий список из текста до первого подзаголовка
        results = [(titles, section_text)]
        for subsection in section.get_sections(levels=[len(titles) + 1]):
            results.extend(
                # Вызываем функцию получения вложенных секций для заданной секции
                all_subsections_from_section(subsection, titles, sections_to_ignore)
                )  # Объединяем результирующие списки данной функции и вызываемой
        return results

In [13]:
def all_subsections_from_title(
    title: str, # Заголовок статьи Википедии, которую парсим
    sections_to_ignore: Set[str] = SECTIONS_TO_IGNORE, # Секции, которые игнорируем
    site_name: str = WIKI_SITE, # Ссылка на сайт википедии
) -> List[Tuple[List[str], str]]:
    """
    Из заголовка страницы Википедии возвращает список всех вложенных секций.
    Каждый подраздел представляет собой кортеж, где:
      - первый элемент представляет собой список родительских секций, начиная с заголовка страницы
      - второй элемент представляет собой текст секции
    """

    # Инициализация объекта MediaWiki
    # WIKI_SITE ссылается на англоязычную часть Википедии
    site = mwclient.Site(site_name)

    # Запрашиваем страницу по заголовку
    page = site.pages[title]

    # Получаем текстовое представление страницы
    text = page.text()

    # Удобный парсер для MediaWiki
    parsed_text = mwparserfromhell.parse(text)
    # Извлекаем заголовки
    headings = [str(h) for h in parsed_text.filter_headings()]
    if headings: # Если заголовки найдены
        # В качестве резюме берем текст до первого заголовка
        summary_text = str(parsed_text).split(headings[0])[0]
    else:
        # Если нет заголовков, то весь текст считаем резюме
        summary_text = str(parsed_text)
    results = [([title], summary_text)] # Добавляем резюме в результирующий список
    for subsection in parsed_text.get_sections(levels=[2]): # Извлекаем секции 2-го уровня
        results.extend(
            # Вызываем функцию получения вложенных секций для заданной секции
            all_subsections_from_section(subsection, [title], sections_to_ignore)
        ) # Объединяем результирующие списки данной функции и вызываемой
    return results

TypeError: 'set' object is not subscriptable

In [23]:
# Разбивка статей на секции
# придется немного подождать, так как на парсинг 100 статей требуется около минуты
wikipedia_sections = []
for idx, title in enumerate(titles):
    print(idx, title)
    wikipedia_sections.extend(all_subsections_from_title(title))
print(f"Найдено {len(wikipedia_sections)} секций на {len(titles)} страницах")

0 NZ Performance Car
1 SsangYong WZ
2 Pac-car II
3 Head-up display
4 Rebadging
5 Vehicle frame
6 Technischer Überwachungsverein
7 DNV Fuel Fighter
8 1950s American automobile culture
9 EUKOR
10 Riversimple Urban Car
11 Grimaldi Group
12 Quill drive
13 UW Hybrid Vehicle Team
14 AMC Rambler Tarpon
15 Fleet special
16 Yamaha OX99-11
17 Japanese Automotive Standards Organization
18 Automotive aftermarket
19 ULTRA AP
20 Mallika Srinivasan
21 Underseal
22 Body solder
23 List of countries by motor vehicle production in the 2010s
24 Experimental safety vehicle
25 Ford Mighty F-350
26 Steam Automobile Club of America
27 QS9000
28 Novo (car)
29 Dogo SS-2000
30 Capitalism: A Love Story
31 General Motors of Canada Ltd v City National Leasing
32 Electronic parking brake
33 Millbrook Proving Ground
34 Retail floorplan
35 Benjamin Briscoe
36 Robotaxi
37 Rolling code
38 United European Car Carriers
39 Glossary of automotive design
40 Hyundai Glovis
41 Automobile magazine
42 MERCON
43 Paxton Phoenix
44

In [24]:
# Очистка текста секции от ссылок <ref>xyz</ref>, начальных и конечных пробелов
def clean_section(section: Tuple[List[str], str]) -> Tuple[List[str], str]:
    titles, text = section
    # Удаляем ссылки
    text = re.sub(r"<ref.*?</ref>", "", text)
    # Удаляем пробелы вначале и конце
    text = text.strip()
    return (titles, text)

In [25]:

# Применим функцию очистки ко всем секциям с помощью генератора списков
wikipedia_sections = [clean_section(ws) for ws in wikipedia_sections]

In [28]:

# Отфильтруем короткие и пустые секции
def keep_section(section: Tuple[List[str], str]) -> bool:
    """Возвращает значение True, если раздел должен быть сохранен, в противном случае значение False."""
    titles, text = section
    # Фильтруем по произвольной длине, можно выбрать и другое значение
    if len(text) < 16:
        return False
    else:
        return True


original_num_sections = len(wikipedia_sections)
wikipedia_sections = [ws for ws in wikipedia_sections if keep_section(ws)]
print(f"Отфильтровано {original_num_sections-len(wikipedia_sections)} секций, осталось {len(wikipedia_sections)} секций.")

Отфильтровано 0 секций, осталось 5225 секций.


In [32]:
for ws in wikipedia_sections[:5]:
    print(ws[0])
    display(ws[1][:50] + "...")
    print()

['NZ Performance Car']


'{{Use dmy dates|date=December 2019}}\n{{Use New Zea...'


['NZ Performance Car', '==Change in focus==']


"The magazine's focus has evolved from initially co..."


['NZ Performance Car', '==Change in focus==', '===Typical cars featured in NZ Performance Car===']


'*[[Nissan Skyline GT-R]]\n*[[Nissan Silvia]]\n*[[Nis...'


['NZ Performance Car', '==Magazine contents==', '===Modified car features===']


'The extent that a car is required to be modified h...'


['NZ Performance Car', '==Magazine contents==', '===New car features===']


'Less common are new car road tests. Recent example...'

In [36]:
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use

# Функция подсчета токенов
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Функция разделения строк
def halved_by_delimiter(string: str, delimiter: str = "\n") -> List[str]:
    """Разделяет строку надвое с помощью разделителя (delimiter), пытаясь сбалансировать токены с каждой стороны."""

    # Делим строку на части по разделителю, по умолчанию \n - перенос строки
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # разделитель не найден
    elif len(chunks) == 2:
        return chunks  # нет необходимости искать промежуточную точку
    else:
        # Считаем токены
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        # Предварительное разделение по середине числа токенов
        best_diff = halfway
        # В цикле ищем какой из разделителей, будет ближе всего к best_diff
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        # Возвращаем левую и правую часть оптимально разделенной строки
        return [left, right]


# Функция обрезает строку до максимально разрешенного числа токенов
def truncated_string(
    string: str, # строка
    model: str, # модель
    max_tokens: int, # максимальное число разрешенных токенов
    print_warning: bool = True, # флаг вывода предупреждения
) -> str:
    """Обрезка строки до максимально разрешенного числа токенов."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    # Обрезаем строку и декодируем обратно
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Предупреждение: Строка обрезана с {len(encoded_string)} токенов до {max_tokens} токенов.")
    # Усеченная строка
    return truncated_string

# Функция делит секции статьи на части по максимальному числу токенов
def split_strings_from_subsection(
    subsection: Tuple[List[str], str], # секции
    max_tokens: int = 1000, # максимальное число токенов
    model: str = GPT_MODEL, # модель
    max_recursion: int = 5, # максимальное число рекурсий
) -> List[str]:
    """
    Разделяет секции на список из частей секций, в каждой части не более max_tokens.
    Каждая часть представляет собой кортеж родительских заголовков [H1, H2, ...] и текста (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # Если длина соответствует допустимой, то вернет строку
    if num_tokens_in_string <= max_tokens:
        return [string]
    # если в результате рекурсия не удалось разделить строку, то просто усечем ее по числу токенов
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # иначе разделим пополам и выполним рекурсию
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]: # Пробуем использовать разделители от большего к меньшему (разрыв, абзац, точка)
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # если какая-либо половина пуста, повторяем попытку с более простым разделителем
                continue
            else:
                # применим рекурсию на каждой половине
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1, # уменьшаем максимальное число рекурсий
                    )
                    results.extend(half_strings)
                return results
    # иначе никакого разделения найдено не было, поэтому просто обрезаем строку (должно быть очень редко)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]

In [37]:
# Делим секции на части
MAX_TOKENS = 1600
wikipedia_strings = []
for section in wikipedia_sections:
    wikipedia_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wikipedia_sections)} секций Википедии поделены на {len(wikipedia_strings)} строк.")

Предупреждение: Строка обрезана с 3852 токенов до 1600 токенов.
Предупреждение: Строка обрезана с 2865 токенов до 1600 токенов.
Предупреждение: Строка обрезана с 2865 токенов до 1600 токенов.
Предупреждение: Строка обрезана с 2867 токенов до 1600 токенов.
Предупреждение: Строка обрезана с 2868 токенов до 1600 токенов.
Предупреждение: Строка обрезана с 2877 токенов до 1600 токенов.
Предупреждение: Строка обрезана с 2857 токенов до 1600 токенов.
Предупреждение: Строка обрезана с 2872 токенов до 1600 токенов.
Предупреждение: Строка обрезана с 2863 токенов до 1600 токенов.
5225 секций Википедии поделены на 5384 строк.


In [38]:
# Напечатаем пример строки
print(wikipedia_strings[1])

NZ Performance Car

==Change in focus==

The magazine's focus has evolved from initially covering all high performance cars, including Australian [[V8 engine|V8]]s, to [[Japanese Domestic Market|Japanese import cars]], and more recently import car culture and associated activities ([[BMX]], [[Freestyle Motocross|FMX]], interviews with local and international musicians, etc.).

Early issues featured cars such as [[Holden Torana]]s alongside [[Mitsubishi]] [[Mitsubishi Galant VR-4|Galant VR4s.]]


In [39]:
from openai import OpenAI
import os
import getpass

EMBEDDING_MODEL = "text-embedding-ada-002"  # Модель токенизации от OpenAI

os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")
client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

# Функция отправки chatGPT строки для ее токенизации (вычисления эмбедингов)
def get_embedding(text, model="text-embedding-ada-002"):

   return client.embeddings.create(input = [text], model=model).data[0].embedding


In [51]:
df = pd.DataFrame({"text": wikipedia_strings})

df['embedding'] = df.text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

SAVE_PATH = "./automobile.csv"
# Сохранение результата
df.to_csv(SAVE_PATH, index=False)

In [52]:
import pickle

file = open('./automobile.pickle', 'wb')
pickle.dump(df, file)
file.close()

In [42]:
df.head()

,text,embedding
0,NZ Performance Car\n\n{{Use dmy dates|date=Dec...,"[-0.02061622031033039, 0.003011121414601803, 0..."
1,NZ Performance Car\n\n==Change in focus==\n\nT...,"[-0.017560184001922607, 0.00253809941932559, 0..."
2,NZ Performance Car\n\n==Change in focus==\n\n=...,"[-0.008404873311519623, 0.009192416444420815, ..."
3,NZ Performance Car\n\n==Magazine contents==\n\...,"[0.0006532236002385616, 0.011562341824173927, ..."
4,NZ Performance Car\n\n==Magazine contents==\n\...,"[-0.01238475926220417, 0.017139919102191925, 0..."


In [44]:
from scipy import spatial  # вычисляет сходство векторов


# Функция поиска
def strings_ranked_by_relatedness(
    query: str, # пользовательский запрос
    df: pd.DataFrame, # DataFrame со столбцами text и embedding (база знаний)
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y), # функция схожести, косинусное расстояние
    top_n: int = 100 # выбор лучших n-результатов
) -> Tuple[List[str], List[float]]: # Функция возвращает кортеж двух списков, первый содержит строки, второй - числа с плавающей запятой
    """Возвращает строки и схожести, отсортированные от большего к меньшему"""

    # Отправляем в OpenAI API пользовательский запрос для токенизации
    query_embedding_response = openai.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )

    # Получен токенизированный пользовательский запрос
    query_embedding = query_embedding_response.data[0].embedding

    # Сравниваем пользовательский запрос с каждой токенизированной строкой DataFrame
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]

    # Сортируем по убыванию схожести полученный список
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)

    # Преобразовываем наш список в кортеж из списков
    strings, relatednesses = zip(*strings_and_relatednesses)

    # Возвращаем n лучших результатов
    return strings[:top_n], relatednesses[:top_n]

In [48]:
strings, relatednesses = strings_ranked_by_relatedness("fast car speed", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.800


'NZ Performance Car\n\n==Magazine contents==\n\n===Modified car features===\n\nThe extent that a car is required to be modified has steadily increased since the first issues. While 300&nbsp;hp was considered to be a huge amount of power when the magazine started, 300&nbsp;kW may not even secure a spot as a feature car, unless the rest of the car is outstanding or unusual.\n\nThis is not to say that NZ Performance Car solely focuses on power, but power figures are part of a more holistic approach considered when modifying a car.'

relatedness=0.791


'NZ Performance Car\n\n==Magazine contents==\n\n===Event coverage===\n\n[[Drag racing]], [[Drifting (motorsport)|drifting]], [[rallying]], [[Super Lap]], Japanese [[Super GT]], [[hillclimbing]], [[touring cars]], auto salon, [[auto show]]s and club events are covered, as long as the racing includes Japanese cars.'

relatedness=0.789


"NZ Performance Car\n\n==Change in focus==\n\nThe magazine's focus has evolved from initially covering all high performance cars, including Australian [[V8 engine|V8]]s, to [[Japanese Domestic Market|Japanese import cars]], and more recently import car culture and associated activities ([[BMX]], [[Freestyle Motocross|FMX]], interviews with local and international musicians, etc.).\n\nEarly issues featured cars such as [[Holden Torana]]s alongside [[Mitsubishi]] [[Mitsubishi Galant VR-4|Galant VR4s.]]"

relatedness=0.788


'NZ Performance Car\n\n==Change in focus==\n\n===Typical cars featured in NZ Performance Car===\n\n*[[Nissan Skyline GT-R]]\n*[[Nissan Silvia]]\n*[[Nissan Pulsar GTI-R]]\n*[[Nissan 300ZX]]\n*[[Nissan 350Z]]\n*[[Nissan Laurel]]\n*[[Subaru WRX]]\n*[[Subaru Legacy]]\n*[[Mitsubishi Lancer Evolution]]\n*[[Honda Integra]]\n*[[Honda Civic]]\n*[[Honda S2000]]\n*[[Toyota Celica]]\n*[[Toyota Supra]]\n*[[Toyota AE86|Toyota Levin AE86]]\n*[[Toyota Altezza]]\n*[[Mazda RX-7]]\n*[[Mazda 323]]\n*[[Mitsubishi Galant VR4]]\n*[[Toyota Hilux]]\n*[[Mazda GTX]]\n*[[Toyota Chaser]]'

relatedness=0.783


'NZ Performance Car\n\n==Magazine contents==\n\n===Driver and celebrity interviews===\n\nLocal and international drivers have been interviewed such as [[Ken Block (rally driver)|Ken Block]], [[Rhys Millen]], [[Mike Whiddett]], [[Sébastien Loeb]], [[Linkin Park]]/[[Fort Minor]] and Nathalie Kelly (from [[Fast n Furious Tokyo Drift]]),'